# Face mask detector built using keras tuner


In [31]:
import numpy as np
import pandas as pd
import glob
import os
import cv2
import seaborn as sns
from keras.layers import Input
from sklearn.preprocessing import OneHotEncoder
from keras.applications import vgg16
from tensorflow.keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.layers.core import Dense, Dropout, Activation,Flatten
from keras.models import Sequential,Model
from tensorflow import keras
from tensorflow.keras import layers
from kerastuner import RandomSearch
from sklearn.preprocessing import LabelBinarizer
import tensorflow as  tf
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix

print("imported")

imported


In [2]:
#######################################   TRAINING DATA   #########################################################

labels = []
images= []
print("strt")


file_paths = glob.glob("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face mask\\dataset\\data\\Face Mask Dataset\\Train\\WithoutMask\\*.png")
for image in file_paths:
    im = cv2.imread(image)
    im.resize((224,224,3))
    images.append(im) #.transpose(1, 0, 2))
    labels.append(0)


print("Enter")
print(np.array(images).shape)
file_paths = glob.glob("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face mask\\dataset\\data\\Face Mask Dataset\\Train\\WithMask\\*.png")
for image in file_paths:
    im = cv2.imread(image)
    im.resize((224,224,3))
    images.append(im) #.transpose(1, 0, 2))
    labels.append(1)

    
print("Enter")
print(np.array(images).shape)


images = np.array(images)
#images = images/255.0

#one hot encoding labels
le = LabelBinarizer()
le.fit(labels)
labels = le.transform(labels)
print("done")

strt
Enter
(5000, 224, 224, 3)
Enter
(10000, 224, 224, 3)
done


In [3]:
#######################################   VALIDATION DATA   #########################################################

labels_val = []
images_val = []
print("strt")


file_paths = glob.glob("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face mask\\dataset\\data\\Face Mask Dataset\\Validation\\WithoutMask\\*.png")
for image in file_paths:
    im = cv2.imread(image)
    im.resize((224,224,3))
    images_val.append(im) #.transpose(1, 0, 2))
    labels_val.append(0)


print("Enter")
print(np.array(images_val).shape)
file_paths = glob.glob("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face mask\\dataset\\data\\Face Mask Dataset\\Validation\\WithMask\\*.png")
for image in file_paths:
    im = cv2.imread(image)
    im.resize((224,224,3))
    images_val.append(im) #.transpose(1, 0, 2))
    labels_val.append(1)

    
print("Enter")
print(np.array(images_val).shape)


images_val = np.array(images_val)
#images = images/255.0

#one hot encoding labels
le = LabelBinarizer()
le.fit(labels_val)
labels_val = le.transform(labels_val)
print("done")

strt
Enter
(400, 224, 224, 3)
Enter
(800, 224, 224, 3)
done


In [4]:
def build_model(hp):
  inputs = tf.keras.Input(shape=(224, 224, 3))
  x = inputs
  for i in range(hp.Int('conv_blocks', 3, 5, default=3)):
    filters = hp.Int('filters_' + str(i), 32, 256, step=32)
    for _ in range(2):
      x = tf.keras.layers.Convolution2D(
        filters, kernel_size=(3, 3), padding='same')(x)
      x = tf.keras.layers.BatchNormalization()(x)
      x = tf.keras.layers.ReLU()(x)
    if hp.Choice('pooling_' + str(i), ['avg', 'max']) == 'max':
      x = tf.keras.layers.MaxPool2D()(x)
    else:
      x = tf.keras.layers.AvgPool2D()(x)
  x = tf.keras.layers.GlobalAvgPool2D()(x)
  x = tf.keras.layers.Dense(
      hp.Int('hidden_size', 30, 100, step=10, default=50),
      activation='relu')(x)
  x = tf.keras.layers.Dropout(
      hp.Float('dropout', 0, 0.5, step=0.1, default=0.5))(x)
  outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

  model = tf.keras.Model(inputs, outputs)
  model.compile(
    optimizer=tf.keras.optimizers.Adam(
      hp.Float('learning_rate', 1e-4, 1e-2, sampling='log')),
    loss='binary_crossentropy', 
    metrics=['accuracy'])
  return model

In [9]:
tuner = RandomSearch(build_model, objective='val_accuracy',project_name = "mask",max_trials=1)
# Display search space overview
tuner.search_space_summary()

INFO:tensorflow:Reloading Oracle from existing project .\mask\oracle.json


In [10]:
tuner.search(images, labels, batch_size=8, epochs=10 , validation_data = (images_val,labels_val))


Train on 10000 samples, validate on 800 samples
Epoch 1/10
 1408/10000 [===>..........................] - ETA: 1:00:00 - loss: 0.6109 - accuracy: 0.750 - ETA: 39:06 - loss: 0.7101 - accuracy: 0.5000  - ETA: 32:12 - loss: 0.7250 - accuracy: 0.416 - ETA: 28:46 - loss: 0.6935 - accuracy: 0.500 - ETA: 26:44 - loss: 0.6700 - accuracy: 0.600 - ETA: 25:28 - loss: 0.6405 - accuracy: 0.645 - ETA: 24:34 - loss: 0.6219 - accuracy: 0.696 - ETA: 24:01 - loss: 0.6111 - accuracy: 0.703 - ETA: 23:29 - loss: 0.5970 - accuracy: 0.722 - ETA: 23:07 - loss: 0.5935 - accuracy: 0.725 - ETA: 22:51 - loss: 0.5908 - accuracy: 0.727 - ETA: 22:38 - loss: 0.5866 - accuracy: 0.729 - ETA: 22:28 - loss: 0.5757 - accuracy: 0.730 - ETA: 22:17 - loss: 0.5674 - accuracy: 0.741 - ETA: 22:09 - loss: 0.5521 - accuracy: 0.758 - ETA: 22:02 - loss: 0.5410 - accuracy: 0.773 - ETA: 21:56 - loss: 0.5386 - accuracy: 0.772 - ETA: 21:51 - loss: 0.5340 - accuracy: 0.784 - ETA: 21:47 - loss: 0.5311 - accuracy: 0.789 - ETA: 21:44 - los

INFO:tensorflow:Oracle triggered exit


In [11]:
tuner.results_summary()


In [13]:
# Retrieve the best model and display its architecture
best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization (BatchNo (None, 224, 224, 64)      256       
_________________________________________________________________
re_lu (ReLU)                 (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 224, 224, 64)      36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 224, 224, 64)      256       
_________________________________________________________________
re_lu_1 (ReLU)               (None, 224, 224, 64)      0     

In [15]:

inputs = tf.keras.Input(shape=(224, 224, 3))
x = inputs

x = tf.keras.layers.Convolution2D(64, kernel_size=(3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Convolution2D(64, kernel_size=(3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.MaxPool2D()(x)

x = tf.keras.layers.Convolution2D(192, kernel_size=(3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Convolution2D(192, kernel_size=(3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.MaxPool2D()(x)

x = tf.keras.layers.Convolution2D(224, kernel_size=(3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Convolution2D(224, kernel_size=(3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.AvgPool2D()(x)

x = tf.keras.layers.Convolution2D(32, kernel_size=(3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.Convolution2D(32, kernel_size=(3, 3), padding='same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.ReLU()(x)
x = tf.keras.layers.AvgPool2D()(x)

x = tf.keras.layers.GlobalAvgPool2D()(x)
x = tf.keras.layers.Dense(30, activation='relu')(x)   
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)    



model = tf.keras.Model(inputs, outputs)
model.compile(
optimizer=tf.keras.optimizers.Adam(learning_rate = 0.0001),
loss='binary_crossentropy', 
metrics=['accuracy'])
model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
batch_normalization_40 (Batc (None, 224, 224, 64)      256       
_________________________________________________________________
re_lu_40 (ReLU)              (None, 224, 224, 64)      0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
batch_normalization_41 (Batc (None, 224, 224, 64)      256       
_________________________________________________________________
re_lu_41 (ReLU)              (None, 224, 224, 64)      0   

In [17]:
model.fit(images, labels, batch_size=8, epochs=10 , validation_data = (images_val,labels_val),verbose = 2)


Train on 10000 samples, validate on 800 samples
Epoch 1/10
10000/10000 - 1397s - loss: 0.1961 - accuracy: 0.9491 - val_loss: 0.0878 - val_accuracy: 0.9737
Epoch 2/10
10000/10000 - 1314s - loss: 0.1040 - accuracy: 0.9717 - val_loss: 0.0735 - val_accuracy: 0.9750
Epoch 3/10
10000/10000 - 1323s - loss: 0.0694 - accuracy: 0.9798 - val_loss: 0.0368 - val_accuracy: 0.9900
Epoch 4/10
10000/10000 - 1305s - loss: 0.0612 - accuracy: 0.9826 - val_loss: 0.1457 - val_accuracy: 0.9438
Epoch 5/10
10000/10000 - 1391s - loss: 0.0514 - accuracy: 0.9856 - val_loss: 0.0351 - val_accuracy: 0.9925
Epoch 6/10
10000/10000 - 1370s - loss: 0.0490 - accuracy: 0.9851 - val_loss: 0.0784 - val_accuracy: 0.9800
Epoch 7/10
10000/10000 - 1298s - loss: 0.0465 - accuracy: 0.9857 - val_loss: 0.0289 - val_accuracy: 0.9937
Epoch 8/10
10000/10000 - 1444s - loss: 0.0367 - accuracy: 0.9892 - val_loss: 0.0282 - val_accuracy: 0.9937
Epoch 9/10
10000/10000 - 1298s - loss: 0.0362 - accuracy: 0.9892 - val_loss: 0.0316 - val_accura

In [19]:
model_json = model.to_json()
with open("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face mask\\Model\\model_json.json", "w") as json_file:
    json_file.write(model_json)
#model.save_model("C:\\MyLearnings\\DeepLearning\\Object Detection\\Projects\\FacialEmotionDetection\\Datasets\\fer2013_images\\fer_images_classes\\Model\\saved_model.h5",save_format='h5')
model.save_weights("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face mask\\Model\\model.h5")
print("saved")

saved


In [20]:
########################################## Evaluating on test set  #############################################


labels = []
images= []
print("strt")


file_paths = glob.glob("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face mask\\dataset\\data\\Face Mask Dataset\\Test\\WithoutMask\\*.png")
for image in file_paths:
    im = cv2.imread(image)
    im.resize((224,224,3))
    images.append(im) #.transpose(1, 0, 2))
    labels.append(0)


print("Enter")
print(np.array(images).shape)
file_paths = glob.glob("C:\\MyLearnings\\DeepLearning\\Facial Projects\\Face mask\\dataset\\data\\Face Mask Dataset\\Test\\WithMask\\*.png")
for image in file_paths:
    im = cv2.imread(image)
    im.resize((224,224,3))
    images.append(im) #.transpose(1, 0, 2))
    labels.append(1)

    
print("Enter")
print(np.array(images).shape)


images = np.array(images)
#images = images/255.0

#one hot encoding labels
le = LabelBinarizer()
le.fit(labels)
labels = le.transform(labels)
print("done")

strt
Enter
(509, 224, 224, 3)
Enter
(992, 224, 224, 3)
done


In [21]:
scores = model.evaluate(images, labels, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


accuracy: 99.09%
